In [3]:
# SLUG MODEL TRANSLATD FROM MATLAB TO PYTHON 
# I had to do this because I could not understanD the errors I kept encountering using MATLAB ;( 

import numpy as np

def slug_model(pars):
    """
    Run the slug model and calculate the sum of squared residuals (SSR).

    Parameters:
    pars (list or array-like): List containing two parameters [S, T].

    Returns:
    float: Sum of squared residuals (SSR).
    """
    # Define the time of the observations
    t = np.array([5.0, 10.0, 20.0, 30.0, 40.0, 50.0])

    # Define the distance from injection
    d = 10

    # Define the amount of injected water
    Q = 50

    # Extract the storage (S) and transmissivity (T) from parameters
    S, T = pars

    # Predict "h" using the slug model equation
    h_pred = (Q / (4 * np.pi * T * t)) * np.exp((-d ** 2 * S) / (4 * T * t))

    
    # Define the measured head
    h_obs = np.array([0.55, 0.47, 0.30, 0.22, 0.17, 0.14])

    # Calculate residuals
    res = h_pred - h_obs

    # Calculate sum of squared residuals (SSR)
    SSR = np.dot(res, res)

    return SSR


# interceptmodel 
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import scipy.io

# Load measurement data from a .mat file
data = scipy.io.loadmat('/dfs6/pub/gshalgum/CEE290AHW/measurement.mat')

# Extract data
obsTime = data['obsTime'].flatten()
obsPotEvap = data['obsPotEvap']
obsPrecip = data['obsPrecip']
obsStorage = data['obsStorage'].flatten()

# Interception Model function
def interceptionModel(t, S, P, E0, a, b, c, d):
    # Interpolators
    P_interp = interp1d(P[:, 0], P[:, 1], bounds_error=False, fill_value="extrapolate")
    E0_interp = interp1d(E0[:, 0], E0[:, 1], bounds_error=False, fill_value="extrapolate")

    # Interpolated values
    P_int = P_interp(t)
    E0_int = E0_interp(t)

    # Model calculations
    I = a * P_int
    D = b * np.maximum(S - c, 0)
    E = np.nan_to_num(d * E0_int * (S / c), nan=0.0) if c != 0 else 0

    # Change in storage
    dSdt = I - D - E
    return dSdt

# Residuals function for the Interception Model
def residuals(params, obsTime, obsPrecip, obsPotEvap, obsStorage):
    # Define a wrapper for solve_ivp
    def model(t, S):
        return interceptionModel(t, S, obsPrecip, obsPotEvap, *params)

    # Solve the differential equations for the full observation period
    sol = solve_ivp(model, [obsTime[0], obsTime[-1]], [obsStorage[0]], t_eval=obsTime, vectorized=True)
    simulatedStorage = sol.y.flatten()
    return obsStorage - simulatedStorage

# Objective function to minimize: sum of squared residuals
def interception_objective(params):
    residuals_vector = residuals(params, obsTime, obsPrecip, obsPotEvap, obsStorage)
    return np.dot(residuals_vector, residuals_vector)



/opt/apps/python/3.10.2/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
import numpy as np

# Function to apply reflection to keep parameters within bounds
def reflect_params(params, bounds):
    reflected_params = np.array(params)
    for i, (low, high) in enumerate(bounds):
        if reflected_params[i] < low:
            reflected_params[i] = low + abs(reflected_params[i] - low)
        elif reflected_params[i] > high:
            reflected_params[i] = high - abs(reflected_params[i] - high)
    return reflected_params

# General Differential Evolution implementation
def G_differential_evolution(objective_func, bounds, pop_size=20, generations=50, F=0.8, CR=0.9):
    """
    General Differential Evolution (DE) optimization algorithm with reflection.

    Parameters:
    objective_func (callable): Objective function to minimize.
    bounds (list of tuples): Bounds for each parameter in the form [(min1, max1), (min2, max2), ...].
    pop_size (int): Number of individuals in the population.
    generations (int): Number of generations to run.
    F (float): Mutation factor.
    CR (float): Crossover rate.

    Returns:
    tuple: Optimized parameters and fitness progression.
    """
    # Initialize population randomly within the given bounds
    population = np.array([
        [np.random.uniform(low, high) for low, high in bounds]
        for _ in range(pop_size)
    ])

    # Evaluate fitness for the initial population
    fitness = np.array([objective_func(ind) for ind in population])

    # Initialize list to store best fitness scores for plotting
    best_fitness_progress = []

    # DE algorithm main loop
    for gen in range(generations):
        for i in range(pop_size):
            # Mutation: select three distinct random indices different from i
            idxs = np.random.choice(np.delete(np.arange(pop_size), i), 3, replace=False)
            a, b, c = population[idxs]
            mutant = a + F * (b - c)

            # Apply reflection for out-of-bound parameters
            mutant = reflect_params(mutant, bounds)

            # Crossover: create a trial vector
            trial = np.array([mutant[j] if np.random.rand() < CR else population[i, j] for j in range(len(bounds))])

            # Selection: evaluate the trial vector
            trial_fitness = objective_func(trial)

            # Replace parent if the trial vector has better fitness
            if trial_fitness < fitness[i]:
                population[i] = trial
                fitness[i] = trial_fitness

        # Record the best fitness of the current generation
        best_idx = np.argmin(fitness)
        best_fitness_progress.append(fitness[best_idx])
        print(f"Generation {gen + 1}, Best Fitness: {fitness[best_idx]:.4f}")

    # Return the best solution and fitness progression
    best_idx = np.argmin(fitness)
    return population[best_idx], best_fitness_progress


# Slug Model parameters
def slug_model(params):
    t = np.array([5.0, 10.0, 20.0, 30.0, 40.0, 50.0])
    d = 10
    Q = 50
    S, T = params

    h_pred = (Q / (4 * np.pi * T * t)) * np.exp((-d ** 2 * S) / (4 * T * t))
    h_obs = np.array([0.55, 0.47, 0.30, 0.22, 0.17, 0.14])

    res = h_pred - h_obs
    return np.dot(res, res)

# Objective function wrapper for the Slug Model
def slug_objective(params):
    return slug_model(params)

# Define bounds for Slug Model parameters [S, T]
slug_bounds = [(0, 1), (0, 2)]

# Run the Differential Evolution algorithm with reflection for the Slug Model
optimal_params_slug, _ = G_differential_evolution(slug_objective, slug_bounds)
print(f"Optimized Slug Model Parameters (DE): {optimal_params_slug}")


Generation 1, Best Fitness: 0.0475
Generation 2, Best Fitness: 0.0475
Generation 3, Best Fitness: 0.0475
Generation 4, Best Fitness: 0.0372
Generation 5, Best Fitness: 0.0005
Generation 6, Best Fitness: 0.0005
Generation 7, Best Fitness: 0.0005
Generation 8, Best Fitness: 0.0005
Generation 9, Best Fitness: 0.0005
Generation 10, Best Fitness: 0.0005
Generation 11, Best Fitness: 0.0005
Generation 12, Best Fitness: 0.0005
Generation 13, Best Fitness: 0.0004
Generation 14, Best Fitness: 0.0004
Generation 15, Best Fitness: 0.0000
Generation 16, Best Fitness: 0.0000
Generation 17, Best Fitness: 0.0000
Generation 18, Best Fitness: 0.0000
Generation 19, Best Fitness: 0.0000
Generation 20, Best Fitness: 0.0000
Generation 21, Best Fitness: 0.0000
Generation 22, Best Fitness: 0.0000
Generation 23, Best Fitness: 0.0000
Generation 24, Best Fitness: 0.0000
Generation 25, Best Fitness: 0.0000
Generation 26, Best Fitness: 0.0000
Generation 27, Best Fitness: 0.0000
Generation 28, Best Fitness: 0.0000
G

In [ ]:
# Objective function to minimize: sum of squared residuals
def interception_objective(params):
    residuals_vector = residuals(params, obsTime, obsPrecip, obsPotEvap, obsStorage)
    return np.dot(residuals_vector, residuals_vector)

# Set the bounds for parameters [a, b, c, d]
interception_bounds = [(0, 1), (0, 2), (0, 10), (0, 1)]

# General Differential Evolution implementation (function provided above)
optimal_params_interception, _ = G_differential_evolution(interception_objective, interception_bounds)
print(f"Optimized Interception Model Parameters (DE): {optimal_params_interception}")

Generation 1, Best Fitness: 8.4669
Generation 2, Best Fitness: 8.4669
Generation 3, Best Fitness: 3.7156
Generation 4, Best Fitness: 3.7156
Generation 5, Best Fitness: 2.5826
Generation 6, Best Fitness: 2.5826
Generation 7, Best Fitness: 1.8471
Generation 8, Best Fitness: 1.8471
Generation 9, Best Fitness: 1.8471
